In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from helpers import *

# pandas formatting
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', 200)

In [2]:
csas_requests = sql("SELECT * FROM csas2_csasrequest")
request_reviews = sql("SELECT * FROM csas2_csasrequestreview")
request_reviewers = sql("SELECT * FROM csas2_requestreviewer")
request_notes = sql("SELECT * FROM csas2_csasrequestnote")
processes = sql("SELECT * FROM csas2_process")
process_notes = sql("SELECT * FROM csas2_processnote")
tors = sql("SELECT * FROM csas2_termsofreference")
tor_reviewers = sql("SELECT * FROM csas2_torreviewer")
meetings = sql("SELECT * FROM csas2_meeting")
meeting_note = sql("SELECT * FROM csas2_meetingnote")
documents = sql("SELECT * FROM csas2_document")
document_notes = sql("SELECT * FROM csas2_documentnote")
document_tracking = sql("SELECT * FROM csas2_documenttracking")

all_tables = [csas_requests, request_reviews, request_reviewers, request_notes, processes, process_notes, 
              tors, tor_reviewers, meetings, meeting_note, documents, document_notes, document_tracking]


In [3]:
def find_var_name(var):
    for name, value in globals().items():
        if value is var:
            return name
    return None

## Virtual Meetings

In [54]:
meetings.loc[(meetings.is_virtual == 0) & (meetings.location.str.contains('virtual', case=False)), ['id', 'name', 'location', 'is_virtual']]

,id,name,location,is_virtual
2,4,Validation of DO as MEQ measure of nutrient loading status of estuaries,Virtual,0
4,8,Status of Atlantic salmon in Gulf Region and assessment of reference points and decision rules that conform to the precautionary approach,Virtual,0
122,162,Stock Assessment of Capelin in Divisions 2J+3KL,"St. John's, NL and Virtual",0
179,226,Recovery Potential Assessment – Northern Brook Lamprey (Ichthyomyzon fossor) – Saskatchewan-Nelson River Population,"Winnipeg, MB and Virtual Meeting",0
223,296,Update of Stock Status for Lobster in Lobster Fishing Area 33,Virtual,0
226,299,Update of Stock Status for Lobster in Lobster Fishing Area 41,Virtual,0
277,360,A data synopsis for British Columbia groundfish: 2021 update,Virtual,0
293,381,Steering Committee Meeting,Virtual,0
310,400,Technical Review of Project-Specific Drill Cutting Dispersion Modelling for Tilt Cove Exploration Drilling Project Environmental Impact Statement,"St. John's, NL and Virtual",0
322,412,Identification of Vulnerable Marine Ecosystems on Seamounts in the North Pacific Fisheries Commission Convention Area using Visual Surveys and Distribution Models,Virtual Meeting,0


## Inconsistent Links

In [44]:
english_columns = ['url_en', 'library_link_en']
french_columns = ['url_fr', 'library_link_fr']

english_mask = documents[english_columns].apply(lambda col: col.str.contains("fra", case=False)).any(axis=1)
french_mask = documents[french_columns].apply(lambda col: col.str.contains("eng", case=False)).any(axis=1)

for _, row in documents.loc[english_mask | french_mask, ['id', 'title_en'] + english_columns + french_columns].iterrows():
    print(str(row[0])+':', row[1])
    display(pd.DataFrame(row).iloc[2:])

11: Dissolved oxygen as a Marine Environmental Quality (MEQ) measure in upper estuaries of the southern Gulf of St. Lawrence: Implications for nutrient management and eelgrass (Zostera marina) coverage


,7
url_en,http://www.dfo-mpo.gc.ca/csas-sccs/Publications/ResDocs-DocRech/2021/2021_056-fra.html
library_link_en,None
url_fr,http://www.dfo-mpo.gc.ca/csas-sccs/Publications/ResDocs-DocRech/2021/2021_056-fra.html
library_link_fr,None


363: Stock Assessment of NAFO Subdivision 3Ps Cod


,308
url_en,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2017/2017_002-eng.html
library_link_en,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2017/2017_002-eng.html
url_fr,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2017/2017_002-eng.html
library_link_fr,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2017/2017_002-eng.html


408: Recovery Potential Assessment for the American Eel (Anguilla rostrata) for eastern Canada: recovery potential assessment population modelling


...

,354
url_en,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/ResDocs-DocRech/2013/2013_132-eng.html
library_link_en,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/ResDocs-DocRech/2013/2013_132-eng.html
url_fr,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/ResDocs-DocRech/2013/2013_132-eng.html
library_link_fr,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/ResDocs-DocRech/2013/2013_132-eng.html


449: Recovery potential assessment of American Eel (Anguilla rostrata) in eastern Canada


,393
url_en,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2013/2013_078-eng.html
library_link_en,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2013/2013_078-eng.html
url_fr,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2013/2013_078-eng.html
library_link_fr,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2013/2013_078-eng.html


812:  Assessment of the risk to Fraser River Sockeye Salmon due to viral haemorrhagic septicaemia virus IVa (VHSV-IVa) transfer from Atlantic Salmon farms in the Discovery Islands area, British Columbia


,712
url_en,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/ResDocs-DocRech/2020/2020_065-eng.htmlC:\Users\CHOWDHURYN\AppData\Roaming\OpenText\DM\Temp\MECTS-4208621-v17-2020_SRS_ABAAHS_CSAS_2020_065_NCR_Asse...
library_link_en,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/ResDocs-DocRech/2020/2020_065-eng.htmlC:\Users\CHOWDHURYN\AppData\Roaming\OpenText\DM\Temp\MECTS-4208621-v17-2020_SRS_ABAAHS_CSAS_2020_065_NCR_Asse...
url_fr,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/ResDocs-DocRech/2020/2020_065-fra.htmlC:\Users\CHOWDHURYN\AppData\Roaming\OpenText\DM\Temp\MECTS-4208621-v17-2020_SRS_ABAAHS_CSAS_2020_065_NCR_Asse...
library_link_fr,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/ResDocs-DocRech/2020/2020_065-fra.htmlC:\Users\CHOWDHURYN\AppData\Roaming\OpenText\DM\Temp\MECTS-4208621-v17-2020_SRS_ABAAHS_CSAS_2020_065_NCR_Asse...


## EDA
